## Write all states and a single file for each State in the 1k-15k data range

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import zipfile
import timeit
from IPython.display import clear_output

In [2]:
ODpath = Path("../data/OD/")
OD_file = ODpath.joinpath("od_distance_1k-15k_clean.csv.gz")
if OD_file.exists ():
    print ("OD file exist")
else:
    print ("OD file does not exist")
    

OD file exist


In [3]:
%time df = pd.read_csv(OD_file, compression='gzip', dtype={'w_geocode': str,'h_geocode':str})

Wall time: 2min 21s


#### Group the data to verify max and min values
Should be between near 1k to 15k

In [4]:
df_group = df.groupby(['w_geocode']).sum()
print ('The max summed S000 is:', "{:,}".format(df_group.S000.max()))
print ('The min summed S000 is:', "{:,}".format(df_group.S000.min()))

The max summed S000 is: 17,248
The min summed S000 is: 1,000


#### Identify all state FIPS codes in data

In [5]:
state_list = df['w_geocode'].str.slice(0,2).unique().tolist()
print ("{:,}".format(len(state_list)))

48


#### Create a column to contain the state FIPS

In [6]:
df['state'] = df['w_geocode'].str.slice(0,2)
print ('Number of records in dataframe:', len(df))
df.head()

Number of records in dataframe: 28328202


,w_geocode,h_geocode,distance,w_group_count,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,SI02,SI03,w_lat,w_lon,h_lat,h_lon,state
0,010010205001001,010010201001027,6962,1119,1,0,0,1,1,0,0,0,1,0,32.457,-86.415,32.452,-86.489,01
1,010010205001001,010010201002005,8132,1119,1,1,0,0,0,1,0,0,0,1,32.457,-86.415,32.493,-86.490,01
2,010010205001001,010010201002006,7700,1119,1,1,0,0,1,0,0,0,0,1,32.457,-86.415,32.482,-86.491,01
3,010010205001001,010010201002016,7088,1119,2,0,1,1,0,1,1,0,2,0,32.457,-86.415,32.474,-86.488,01
4,010010205001001,010010201002017,6830,1119,4,2,1,1,1,1,2,0,3,1,32.457,-86.415,32.477,-86.484,01


#### Filter the data to include commute distances > 15 miles and <60 miles

In [7]:
df_filter = df[(df['distance'] > 24000) & (df['distance'] <= 100000)].copy()
print ('Number of records after filter for 15-60 mile commutes:', "{:,}".format(len(df_filter)))

Number of records after filter for 15-60 mile commutes: 8,784,835


#### Reset the w_group_count
** This is needed to correct an issue caused by chunking the data in an earlier step

In [8]:
df_w_counts = pd.DataFrame(df,columns=['w_geocode','w_group_count'])

In [9]:
df_w_counts_unique = df_w_counts.drop_duplicates()

In [10]:
df_w_count_group = df_w_counts_unique.groupby(['w_geocode']).sum()
df_w_count_group = df_w_count_group.reset_index()
df_w_count_group.head()

,w_geocode,w_group_count
0,010010205001001,1119
1,010030107032109,1048
2,010030112023027,1159
3,010030115021041,1460
4,010059505002038,1004


#### Merge the reset counts with the dataframe

In [11]:
merge_result = pd.merge(df_filter, df_w_count_group, left_on=  ['w_geocode'],right_on= ['w_geocode'],how = 'left')

merge_result.rename(columns={'w_group_count_y': 'w_group_count'}, inplace=True)
# Remove duplicate column names with _x suffix
merge_result.drop(list(merge_result.filter(regex='_x')), axis=1, inplace=True)
merge_result.head()

,w_geocode,h_geocode,distance,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,SI02,SI03,w_lat,w_lon,h_lat,h_lon,state,w_group_count
0,010010205001001,010010209001001,30634,1,1,0,0,1,0,0,0,0,1,32.457,-86.415,32.705,-86.559,01,1119
1,010010205001001,010010209001002,29897,2,2,0,0,1,1,0,0,0,2,32.457,-86.415,32.694,-86.567,01,1119
2,010010205001001,010010209001015,31828,1,1,0,0,0,1,0,0,0,1,32.457,-86.415,32.699,-86.597,01,1119
3,010010205001001,010010209001024,29513,2,0,1,1,1,1,0,0,1,1,32.457,-86.415,32.653,-86.627,01,1119
4,010010205001001,010010209002004,25250,1,0,1,0,0,0,1,0,0,1,32.457,-86.415,32.665,-86.522,01,1119


#### With updated counts filter for those above 15,000 employees per block

In [12]:
df_filter = merge_result[(merge_result['w_group_count'] < 15000)].copy()

#### Write the updated file to compressed CSV

In [13]:
output = 'od_distance_1k-15k_15-60_miles.csv'
out_file_path = ODpath.joinpath(output)
df_filter.to_csv(out_file_path, index=None)
print ('Number of records written:', "{:,}".format(len(df_filter)))

Number of records written: 8,770,266


#### Funtion to write each state file to csv

In [14]:
def write_state(df_state):
    df_state.to_csv(out_state, index=None)

#### Loop through each State FIPS code and call function to write csv

In [15]:
for index in range(0,len(state_list)):
    state_id = state_list[index]
    df_state = df_filter[df_filter['state'] == state_id]
    outputstate = state_id + '_od_distance_1k-15k_15-60_miles.csv'
    out_state = ODpath.joinpath(outputstate)
    write_state(df_state)

#### Limit data to continental U.S.
Filter out Alaska and Hawaii

In [16]:
df_filter_continental = df_filter[(~df_filter['state'].isin(['02','15']))].copy()
df_filter_continental.state.nunique()

48

#### Write the continental file to compressed CSV

In [17]:
output = 'od_distance_1k-15k_15-60_miles_continental.csv'
out_file_path = ODpath.joinpath(output)
df_filter_continental.to_csv(out_file_path, index=None)
print ('Number of records written:', "{:,}".format(len(df_filter_continental)))

Number of records written: 8,770,266


### END